In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

In [ ]:
page_x_inches: float = 11. # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
p = gp.get_random_point_in_polygon(drawbox)

In [ ]:
drawbox.bounds

In [ ]:
MultiPoint(start_points)

In [ ]:
xcs, ycs = gp.overlay_grid(drawbox.buffer(-30), xstep=25, ystep=25)
start_points = [Point(x,y) for x,y in itertools.product(xcs, ycs)]

In [ ]:
grams = []
for p in tqdm(start_points):
    width = 2
    height = 2
    sq = gp.centered_box(p, width=width, height=height)

    angle = np.random.uniform(360)
    sq = sa.rotate(sq, angle)
    trans_angle = np.random.uniform(np.pi*2)
    l = sq.boundary
    d_buffer = np.random.uniform(0.2, 0.4)
    d_translate = 0.1
    ls = []
    breakpoint=np.random.uniform(0, 0.6)
    for i in range(40):
        sl = so.substring(l, 0., breakpoint, normalized=True)
        jsl = gp.LineString_to_jittered_bezier(sl, n_midpoints=1, xstd=0.015, ystd=0.015, normalized=True)
        rsl = so.substring(l, breakpoint, 1, normalized=True)
        coords = list(jsl.coords) + list(rsl.coords)
        sq = Polygon(coords).buffer(d_buffer, cap_style=2, join_style=2)
        xoff = np.cos(trans_angle) * d_translate
        yoff = np.sin(trans_angle) * d_translate
        sq = sa.translate(sq, xoff=xoff, yoff=yoff)
        l = sq.boundary
        ls.append(l)
    gram = MultiLineString(ls)
    grams.append(gram)

In [ ]:
layer = gp.merge_LineStrings(grams)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(layer)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0103_substring_jitter.svg'

sk.save(savepath)

In [ ]:
vpype_commands = 'reloop linesimplify --tolerance 0.01mm linemerge --tolerance 0.1mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

In [ ]:
p = drawbox.centroid
width = 20
height = 20
c = gp.centered_box(p, width=width, height=height)
c = sa.rotate(c, angle=np.random.uniform(360))
l = c.boundary
ls = []


In [ ]:
sl.type

In [ ]:
sl = so.substring(l, 0., breakpoint, normalized=True)
jsl = gp.LineString_to_jittered_bezier(sl, n_midpoints=2, xstd=0.001, ystd=0.002, normalized=True)
jsl

In [ ]:
rsl = so.substring(l, breakpoint, 1, normalized=True)

In [ ]:

d_buffer = 0.25
d_translate=0.15
breakpoint = np.random.uniform(0.15,0.85)
angle = np.random.uniform(0, np.pi*2)
for i in range(1):
    sl = so.substring(l, 0., breakpoint, normalized=True)
    jsl = gp.LineString_to_jittered_bezier(sl, n_midpoints=2, xstd=0.001, ystd=0.002, normalized=True)
    rsl = so.substring(l, breakpoint, 1, normalized=True)
    coords = list(jsl.coords) + list(rsl.coords)
    c = Polygon(coords).buffer(d_buffer, cap_style=2, join_style=2)
    xoff = np.cos(angle) * d_translate
    yoff = np.sin(angle) * d_translate
    c = sa.translate(c, xoff=xoff, yoff=yoff)
    l = c.boundary
    ls.append(l)
layer = MultiLineString(ls)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(layer)
sk.display(color_mode='none')

In [ ]:
ls = LineString([(50,50), (50, 10)])

In [ ]:
jls = gp.LineString_to_jittered_bezier(ls, n_midpoints=15, xbias=0., xstd=0.03, ybias=0., ystd=0., normalized=True, n_eval_points=50)

In [ ]:
n_points = len(jls.coords)

In [ ]:
gp.gaussian_random_walk(100, )

In [ ]:
curve1 = bezier.Curve(np.asfortranarray(jitter_ls).T, degree=n_midpoints+1)
bez = curve1.evaluate_multi(np.linspace(0., 1., n_eval_points))
sg.asLineString(bez.T)